# Research experiments for the purposes of the master's thesis titled "Artificial intelligence methods in fall detection based on accelerometer data"

# Check GPU availability

In [ ]:
from tensorflow.python.client import device_lib


def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

# Import necessary libraries

In [ ]:
import scipy.io
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import numpy as np

# Classification of falls within other activities - fall data vs ADL data

## Data preprocessing

In [ ]:
fall_data = scipy.io.loadmat('./UniMiB-SHAR/data/fall_data.mat')['fall_data']
adl_data  = scipy.io.loadmat('./UniMiB-SHAR/data/adl_data.mat')['adl_data']
fall_labels = np.zeros(fall_data.shape[0])
adl_labels = np.ones(adl_data.shape[0])

data = np.concatenate( (fall_data, adl_data) )
labels = np.concatenate( (fall_labels, adl_labels) )

x = data.reshape(data.shape[0], data.shape[1], 1)
y = labels

classes = [0, 1]
NUMBER_OF_CLASSES = len(classes)

In [ ]:
class_idxs = {cl: np.where(y == cl)[0] for cl in classes}
print("Number of samples in each class:")
print([len(class_idxs[i]) for i in classes])
numer_of_test_samples = int(len(x)*0.2)
print("Number of test samples: ", numer_of_test_samples)
print("Number of samples in each class in test set:")
print([numer_of_test_samples//2 for i in classes])
print("Number of samples in each class in train set:")
print([len(class_idxs[i]) - numer_of_test_samples//2 for i in classes])

In [ ]:
# Find indexes of each group
cl_indexes = {}
cl_indexes = {cl:np.where(y==cl)[0] for cl in classes}

# take the the samples to test set (equal number for each class)
number_of_test_samples = int(len(x)*0.2)
number_of_test_samples_per_class = number_of_test_samples // NUMBER_OF_CLASSES

val_idxs = [item for cl, items in cl_indexes.items() for item in items[:number_of_test_samples_per_class]]
train_idxs = [item for cl, items in cl_indexes.items() for item in items[number_of_test_samples_per_class:]]

x_val = x[val_idxs]
y_val = y[val_idxs]

x_train = x[train_idxs]
y_train = y[train_idxs]

## Training on CNN-infused model

In [ ]:
hidden_dim = 512
seq_len = x.shape[1]
n_features = 1
epochs = 20
learning_rate = 0.0003
batch_size = 16

regularizers = {
    'kernel_regularizer':tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
    'bias_regularizer':tf.keras.regularizers.l2(1e-4),
    'activity_regularizer':tf.keras.regularizers.l2(1e-5)
}

in1 = tf.keras.layers.Input(shape=(seq_len, n_features))
gru1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(hidden_dim, return_sequences=True, **regularizers))(in1)
gru2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(hidden_dim, name='latent_layer', **regularizers))(gru1)
rsh1 = tf.keras.layers.Reshape( (hidden_dim*2, 1) )(gru2)
cov1 = tf.keras.layers.Dropout(0.1)(tf.keras.layers.Conv1D(seq_len, 3, activation='relu', **regularizers)(rsh1))
max1 = tf.keras.layers.MaxPool1D(pool_size=3, strides=1)(cov1)
cov2 = tf.keras.layers.Dropout(0.1)(tf.keras.layers.Conv1D(seq_len, 5, activation='relu', **regularizers)(max1))
max2 = tf.keras.layers.MaxPool1D(pool_size=3, strides=1)(cov2)
cov3 = tf.keras.layers.Dropout(0.1)(tf.keras.layers.Conv1D(seq_len, 3, activation='relu', **regularizers)(max2))
max3 = tf.keras.layers.MaxPool1D(pool_size=3, strides=1)(cov3)
rsh2 = tf.keras.layers.Reshape( (seq_len, 1010) )(max3)
tdd1 = tf.keras.layers.Dropout(0.1)(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1,**regularizers))(rsh2))
f1   = tf.keras.layers.Flatten()(tdd1)
d1   = tf.keras.layers.Dense(1, activation='sigmoid', **regularizers)(f1)

classifier3 = tf.keras.Model(
    inputs=[in1], 
    outputs=[d1]
)

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
classifier3.compile(loss='binary_crossentropy', optimizer=opt, 
    metrics=['accuracy'])

history3 = classifier3.fit(
    np.array(x_train), np.array(y_train),
    epochs=epochs,
    steps_per_epoch=x.shape[0] // batch_size,
    validation_data=(x_val, y_val),
    callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.00003)]
)

In [ ]:
plt.plot(history3.history['val_loss'], label='val_loss')
plt.plot(history3.history['loss'], label='loss')
plt.legend()

In [ ]:
plt.plot(history3.history['val_accuracy'], label='val_accuracy')
plt.plot(history3.history['accuracy'], label='accuracy')
plt.legend()

## CNN-infused model with an additional dense layer

In [ ]:
hidden_dim = 512
seq_len = x.shape[1]
n_features = 1
epochs = 20
learning_rate = 0.0003
batch_size = 16

regularizers = {
    'kernel_regularizer':tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
    'bias_regularizer':tf.keras.regularizers.l2(1e-4),
    'activity_regularizer':tf.keras.regularizers.l2(1e-5)
}

in1 = tf.keras.layers.Input(shape=(seq_len, n_features))
gru1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(hidden_dim, return_sequences=True, **regularizers))(in1)
gru2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(hidden_dim, name='latent_layer', **regularizers))(gru1)
rsh1 = tf.keras.layers.Reshape( (hidden_dim*2, 1) )(gru2)
cov1 = tf.keras.layers.Dropout(0.3)(tf.keras.layers.Conv1D(seq_len, 3, activation='relu', **regularizers)(rsh1))
max1 = tf.keras.layers.MaxPool1D(pool_size=3, strides=1)(cov1)
cov2 = tf.keras.layers.Dropout(0.2)(tf.keras.layers.Conv1D(seq_len, 5, activation='relu', **regularizers)(max1))
max2 = tf.keras.layers.MaxPool1D(pool_size=3, strides=1)(cov2)
cov3 = tf.keras.layers.Dropout(0.1)(tf.keras.layers.Conv1D(seq_len, 3, activation='relu', **regularizers)(max2))
max3 = tf.keras.layers.MaxPool1D(pool_size=3, strides=1)(cov3)
rsh2 = tf.keras.layers.Reshape( (seq_len, 1010) )(max3)
tdd1 = tf.keras.layers.Dropout(0.3)(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1,**regularizers))(rsh2))
den1 = tf.keras.layers.Dense(32, **regularizers)(tdd1) # added dense
f1   = tf.keras.layers.Flatten()(den1)
d1   = tf.keras.layers.Dense(1, activation='sigmoid', **regularizers)(f1)

classifier4 = tf.keras.Model(
    inputs=[in1], 
    outputs=[d1]
)

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
classifier4.compile(loss='binary_crossentropy', optimizer=opt, 
    metrics=['accuracy'])

history4 = classifier4.fit(
    np.array(x_train), np.array(y_train),
    epochs=epochs,
    steps_per_epoch=x.shape[0] // batch_size,
    validation_data=(x_val, y_val),
    callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-8)]
)

In [ ]:
plt.plot(history4.history['val_loss'], label='val_loss')
plt.plot(history4.history['loss'], label='loss')
plt.legend()

In [ ]:
plt.plot(history4.history['val_accuracy'], label='val_accuracy')
plt.plot(history4.history['accuracy'], label='accuracy')
plt.legend()

## CNN-infused hyperparameter optimization - no dense layer

In [ ]:
hidden_dim = 512
seq_len = x.shape[1]
n_features = 1
epochs = 20
learning_rate = 0.0003
batch_size = 16

regularizers = {
    'kernel_regularizer':tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
    'bias_regularizer':tf.keras.regularizers.l2(1e-4),
    'activity_regularizer':tf.keras.regularizers.l2(1e-5)
}

in1 = tf.keras.layers.Input(shape=(seq_len, n_features))
gru1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(hidden_dim, return_sequences=True, **regularizers))(in1)
gru2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(hidden_dim, name='latent_layer', **regularizers))(gru1)
rsh1 = tf.keras.layers.Reshape( (hidden_dim*2, 1) )(gru2)
cov1 = tf.keras.layers.Dropout(0.3)(tf.keras.layers.Conv1D(seq_len, 3, activation='relu', **regularizers)(rsh1)) # changed rate
max1 = tf.keras.layers.MaxPool1D(pool_size=3, strides=1)(cov1)
cov2 = tf.keras.layers.Dropout(0.2)(tf.keras.layers.Conv1D(seq_len, 5, activation='relu', **regularizers)(max1)) # changed rate
max2 = tf.keras.layers.MaxPool1D(pool_size=3, strides=1)(cov2)
cov3 = tf.keras.layers.Dropout(0.1)(tf.keras.layers.Conv1D(seq_len, 3, activation='relu', **regularizers)(max2))
max3 = tf.keras.layers.MaxPool1D(pool_size=3, strides=1)(cov3)
rsh2 = tf.keras.layers.Reshape( (seq_len, 1010) )(max3)
tdd1 = tf.keras.layers.Dropout(0.3)(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1,**regularizers))(rsh2)) # changed rate
f1   = tf.keras.layers.Flatten()(tdd1)
d1   = tf.keras.layers.Dense(1, activation='sigmoid', **regularizers)(f1)

classifier5 = tf.keras.Model(
    inputs=[in1], 
    outputs=[d1]
)

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
classifier5.compile(loss='binary_crossentropy', optimizer=opt, 
    metrics=['accuracy'])

history5 = classifier5.fit(
    np.array(x_train), np.array(y_train),
    epochs=epochs,
    steps_per_epoch=x.shape[0] // batch_size,
    validation_data=(x_val, y_val),
    callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-8)] # changed lr and patience
)

In [ ]:
plt.plot(history5.history['val_loss'], label='val_loss')
plt.plot(history5.history['loss'], label='loss')
plt.legend()

In [ ]:
plt.plot(history5.history['val_accuracy'], label='val_accuracy')
plt.plot(history5.history['accuracy'], label='accuracy')
plt.legend()

## Simplified RNN model

In [ ]:
hidden_dim = 512
seq_len = x.shape[1]
n_features = 1
epochs = 20
learning_rate = 0.0003
batch_size = 16

in1 = tf.keras.layers.Input(shape=(seq_len, n_features))
gru1 = tf.keras.layers.GRU(hidden_dim, name='latent_layer')(in1)
bn = tf.keras.layers.BatchNormalization()(gru1)
dense = tf.keras.layers.Dense(128, activation="relu")(bn)
drop = tf.keras.layers.Dropout(0.3)(dense)
d1   = tf.keras.layers.Dense(1, activation='sigmoid')(drop)

classifier_test = tf.keras.Model(
    inputs=[in1], 
    outputs=[d1]
)

print(classifier_test.summary())

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
classifier_test.compile(loss='binary_crossentropy', optimizer=opt, 
    metrics=['accuracy'])

history_test = classifier_test.fit(
    np.array(x_train), np.array(y_train),
    epochs=epochs,
    steps_per_epoch=x.shape[0] // batch_size,
    validation_data=(x_val, y_val)
)

In [ ]:
plt.plot(history_test.history['val_accuracy'], label='val_accuracy')
plt.plot(history_test.history['accuracy'], label='accuracy')
plt.legend()

## Simplified RNN model - hyperparameter optimization

In [ ]:
hidden_dim = 512
seq_len = x.shape[1]
n_features = 1
epochs = 20
learning_rate = 0.0003
batch_size = 16

regularizers = {
    'kernel_regularizer':tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
    'bias_regularizer':tf.keras.regularizers.l2(1e-4),
    'activity_regularizer':tf.keras.regularizers.l2(1e-5)
}

regularizers_GRU = {
    'kernel_regularizer':tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
    'bias_regularizer':tf.keras.regularizers.l2(1e-4),
    'activity_regularizer':tf.keras.regularizers.l2(1e-5),
    'recurrent_regularizer':tf.keras.regularizers.l2(1e-5)
}

in1 = tf.keras.layers.Input(shape=(seq_len, n_features))
gru1 = tf.keras.layers.GRU(hidden_dim, name='latent_layer', **regularizers_GRU)(in1)
bn = tf.keras.layers.BatchNormalization()(gru1)
dense = tf.keras.layers.Dense(128, activation="relu", **regularizers)(bn)
drop = tf.keras.layers.Dropout(0.3)(dense)
d1   = tf.keras.layers.Dense(1, activation='sigmoid', **regularizers)(drop)

classifier_test_opt = tf.keras.Model(
    inputs=[in1], 
    outputs=[d1]
)

print(classifier_test_opt.summary())

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
classifier_test_opt.compile(loss='binary_crossentropy', optimizer=opt, 
    metrics=['accuracy'])

history_test_opt = classifier_test_opt.fit(
    np.array(x_train), np.array(y_train),
    epochs=epochs,
    steps_per_epoch=x.shape[0] // batch_size,
    validation_data=(x_val, y_val),
    callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-8)]
)

In [ ]:
plt.plot(history_test_opt.history['val_accuracy'], label='val_accuracy')
plt.plot(history_test_opt.history['accuracy'], label='accuracy')
plt.legend()

## Simplified RNN model - bigger batch size

In [ ]:
hidden_dim = 512
seq_len = x.shape[1]
n_features = 1
epochs = 20
learning_rate = 0.0003
batch_size = 32

regularizers = {
    'kernel_regularizer':tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
    'bias_regularizer':tf.keras.regularizers.l2(1e-4),
    'activity_regularizer':tf.keras.regularizers.l2(1e-5)
}

regularizers_GRU = {
    'kernel_regularizer':tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
    'bias_regularizer':tf.keras.regularizers.l2(1e-4),
    'activity_regularizer':tf.keras.regularizers.l2(1e-5),
    'recurrent_regularizer':tf.keras.regularizers.l2(1e-5)
}

in1 = tf.keras.layers.Input(shape=(seq_len, n_features))
gru1 = tf.keras.layers.GRU(hidden_dim, name='latent_layer', **regularizers_GRU)(in1)
bn = tf.keras.layers.BatchNormalization()(gru1)
dense = tf.keras.layers.Dense(128, activation="relu", **regularizers)(bn)
drop = tf.keras.layers.Dropout(0.3)(dense)
d1   = tf.keras.layers.Dense(1, activation='sigmoid', **regularizers)(drop)

classifier_test_opt_b = tf.keras.Model(
    inputs=[in1], 
    outputs=[d1]
)

print(classifier_test_opt_b.summary())

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
classifier_test_opt_b.compile(loss='binary_crossentropy', optimizer=opt, 
    metrics=['accuracy'])

history_test_opt_b = classifier_test_opt_b.fit(
    np.array(x_train), np.array(y_train),
    epochs=epochs,
    steps_per_epoch=x.shape[0] // batch_size,
    validation_data=(x_val, y_val),
    callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-8)]
)

In [ ]:
plt.plot(history_test_opt_b.history['val_accuracy'], label='val_accuracy')
plt.plot(history_test_opt_b.history['accuracy'], label='accuracy')
plt.legend()

## Simplified RNN model - smaller batch size

In [ ]:
hidden_dim = 512
seq_len = x.shape[1]
n_features = 1
epochs = 20
learning_rate = 0.0003
batch_size = 8

regularizers = {
    'kernel_regularizer':tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
    'bias_regularizer':tf.keras.regularizers.l2(1e-4),
    'activity_regularizer':tf.keras.regularizers.l2(1e-5)
}

regularizers_GRU = {
    'kernel_regularizer':tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
    'bias_regularizer':tf.keras.regularizers.l2(1e-4),
    'activity_regularizer':tf.keras.regularizers.l2(1e-5),
    'recurrent_regularizer':tf.keras.regularizers.l2(1e-5)
}

in1 = tf.keras.layers.Input(shape=(seq_len, n_features))
gru1 = tf.keras.layers.GRU(hidden_dim, name='latent_layer', **regularizers_GRU)(in1)
bn = tf.keras.layers.BatchNormalization()(gru1)
dense = tf.keras.layers.Dense(128, activation="relu", **regularizers)(bn)
drop = tf.keras.layers.Dropout(0.3)(dense)
d1   = tf.keras.layers.Dense(1, activation='sigmoid', **regularizers)(drop)

classifier_test_opt_s = tf.keras.Model(
    inputs=[in1], 
    outputs=[d1]
)

print(classifier_test_opt_s.summary())

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
classifier_test_opt_s.compile(loss='binary_crossentropy', optimizer=opt, 
    metrics=['accuracy'])

history_test_opt_s = classifier_test_opt_s.fit(
    np.array(x_train), np.array(y_train),
    epochs=epochs,
    steps_per_epoch=x.shape[0] // batch_size,
    validation_data=(x_val, y_val),
    callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-8)]
)

In [ ]:
plt.plot(history_test_opt_s.history['val_accuracy'], label='val_accuracy')
plt.plot(history_test_opt_s.history['accuracy'], label='accuracy')
plt.legend()

## Bidirectional RNN model

In [ ]:
hidden_dim = 512
seq_len = x.shape[1]
n_features = 1
epochs = 20
learning_rate = 0.0003
batch_size = 16

regularizers = {
    'kernel_regularizer':tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
    'bias_regularizer':tf.keras.regularizers.l2(1e-4),
    'activity_regularizer':tf.keras.regularizers.l2(1e-5)
}

regularizers_GRU = {
    'kernel_regularizer':tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
    'bias_regularizer':tf.keras.regularizers.l2(1e-4),
    'activity_regularizer':tf.keras.regularizers.l2(1e-5),
    'recurrent_regularizer':tf.keras.regularizers.l2(1e-5)
}

in1 = tf.keras.layers.Input(shape=(seq_len, n_features))
gru1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(hidden_dim, name='latent_layer', **regularizers_GRU))(in1)
bn = tf.keras.layers.BatchNormalization()(gru1)
dense = tf.keras.layers.Dense(128, activation="relu", **regularizers)(bn)
drop = tf.keras.layers.Dropout(0.3)(dense)
d1   = tf.keras.layers.Dense(1, activation='sigmoid', **regularizers)(drop)

classifier_test_opt_bi = tf.keras.Model(
    inputs=[in1], 
    outputs=[d1]
)

print(classifier_test_opt_bi.summary())

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
classifier_test_opt_bi.compile(loss='binary_crossentropy', optimizer=opt, 
    metrics=['accuracy'])

history_test_opt_bi = classifier_test_opt_bi.fit(
    np.array(x_train), np.array(y_train),
    epochs=epochs,
    steps_per_epoch=x.shape[0] // batch_size,
    validation_data=(x_val, y_val),
    callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-8)]
)

In [ ]:
plt.plot(history_test_opt_bi.history['val_accuracy'], label='val_accuracy')
plt.plot(history_test_opt_bi.history['accuracy'], label='accuracy')
plt.legend()

# Classification of fall data only - distinguishing between 8 types of falls

## Data preprocessing

In [ ]:
data = scipy.io.loadmat('./UniMiB-SHAR/data/fall_data.mat')['fall_data']
labels = scipy.io.loadmat('./UniMiB-SHAR/data/fall_labels.mat')['fall_labels']
classes = np.unique(labels[:, 0])

In [ ]:
NUMBER_OF_CLASSES = len(classes)

batch_size = 1
bs = batch_size * NUMBER_OF_CLASSES # for ballancing

In [ ]:
class_idxs = {cl: np.where(labels[:, 0] == cl)[0] for cl in classes}

In [ ]:
print("Number of samples in each class:")
[len(class_idxs[i]) for i in classes]

In [ ]:
x = data.reshape(data.shape[0], data.shape[1], 1)
y = labels[:, 0]

In [ ]:
label_to_encoding = {
    1: [1, 0, 0, 0, 0, 0, 0, 0],
    2: [0, 1, 0, 0, 0, 0, 0, 0],
    3: [0, 0, 1, 0, 0, 0, 0, 0],
    4: [0, 0, 0, 1, 0, 0, 0, 0],
    5: [0, 0, 0, 0, 1, 0, 0, 0],
    6: [0, 0, 0, 0, 0, 1, 0, 0],
    7: [0, 0, 0, 0, 0, 0, 1, 0],
    8: [0, 0, 0, 0, 0, 0, 0, 1],
    
}

y_encoded = [label_to_encoding[yy] for yy in y]

In [ ]:
# shuffle data
idx_shuffle = np.arange(len(x))
np.random.shuffle(idx_shuffle)
train_idxs = idx_shuffle[:int(len(x)*0.8)]
val_idxs = idx_shuffle[int(len(x)*0.8):]
x_train = np.array(x)[train_idxs]
x_val = np.array(x)[val_idxs]
y_train_encoded = np.array(y_encoded)[train_idxs]
y_val_encoded = np.array(y_encoded)[val_idxs]

In [ ]:
# Find indexes of each group
cl_indexes = {cl:np.where(y==cl)[0] for cl in classes}

# take the the samples to test set (equal number for each class)
number_of_test_samples = int(len(x)*0.2)
number_of_test_samples_per_class = number_of_test_samples // NUMBER_OF_CLASSES

val_idxs = [item for cl, items in cl_indexes.items() for item in items[:number_of_test_samples_per_class]]
train_idxs = [item for cl, items in cl_indexes.items() for item in items[number_of_test_samples_per_class:]]

x_val = x[val_idxs]
y_val = y[val_idxs]
y_val_encoded = np.array(y_encoded)[val_idxs]

x_train = x[train_idxs]
y_train = y[train_idxs]
y_train_encoded = np.array(y_encoded)[train_idxs]

In [ ]:
class_idxs = {cl: np.where(y_train == cl)[0] for cl in classes}
print("Number of samples in each class:")
[len(class_idxs[i]) for i in classes]

In [ ]:
class_idxs = {cl: np.where(y_val == cl)[0] for cl in classes}
print("Number of samples in each class:")
[len(class_idxs[i]) for i in classes]

## Training on optimized CNN-infused model

In [ ]:
hidden_dim = 512
seq_len = x.shape[1]
n_features = 1
epochs = 60
learning_rate = 0.0003
batch_size = 16

regularizers = {
    'kernel_regularizer':tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
    'bias_regularizer':tf.keras.regularizers.l2(1e-4),
    'activity_regularizer':tf.keras.regularizers.l2(1e-5)
}

in1 = tf.keras.layers.Input(shape=(seq_len, n_features))
gru1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(hidden_dim, return_sequences=True, **regularizers))(in1)
gru2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(hidden_dim, name='latent_layer', **regularizers))(gru1)
rsh1 = tf.keras.layers.Reshape( (hidden_dim*2, 1) )(gru2)
cov1 = tf.keras.layers.Dropout(0.3)(tf.keras.layers.Conv1D(seq_len, 3, activation='relu', **regularizers)(rsh1)) # changed rate
max1 = tf.keras.layers.MaxPool1D(pool_size=3, strides=1)(cov1)
cov2 = tf.keras.layers.Dropout(0.2)(tf.keras.layers.Conv1D(seq_len, 5, activation='relu', **regularizers)(max1)) # changed rate
max2 = tf.keras.layers.MaxPool1D(pool_size=3, strides=1)(cov2)
cov3 = tf.keras.layers.Dropout(0.1)(tf.keras.layers.Conv1D(seq_len, 3, activation='relu', **regularizers)(max2))
max3 = tf.keras.layers.MaxPool1D(pool_size=3, strides=1)(cov3)
rsh2 = tf.keras.layers.Reshape( (seq_len, 1010) )(max3)
tdd1 = tf.keras.layers.Dropout(0.3)(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1,**regularizers))(rsh2)) # changed rate
f1   = tf.keras.layers.Flatten()(tdd1)
d1   = tf.keras.layers.Dense(8, activation='softmax', **regularizers)(f1)

classifier_f = tf.keras.Model(
    inputs=[in1], 
    outputs=[d1]
)

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
classifier_f.compile(loss='categorical_crossentropy', optimizer=opt, 
    metrics=['accuracy'])

history_f = classifier_f.fit(
    x_train, y_train_encoded, batch_size=batch_size, epochs=epochs,
    validation_data=(x_val, y_val_encoded),
    callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-8)]
)

In [ ]:
plt.plot(history_f.history['val_accuracy'], label='val_accuracy')
plt.plot(history_f.history['accuracy'], label='accuracy')
plt.legend()

## Training on final Bidirectional RNN model

In [ ]:
hidden_dim = 512
seq_len = x.shape[1]
n_features = 1
epochs = 60
learning_rate = 0.0003
batch_size = 16

regularizers = {
    'kernel_regularizer':tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
    'bias_regularizer':tf.keras.regularizers.l2(1e-4),
    'activity_regularizer':tf.keras.regularizers.l2(1e-5)
}

regularizers_GRU = {
    'kernel_regularizer':tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
    'bias_regularizer':tf.keras.regularizers.l2(1e-4),
    'activity_regularizer':tf.keras.regularizers.l2(1e-5),
    'recurrent_regularizer':tf.keras.regularizers.l2(1e-5)
}

in1 = tf.keras.layers.Input(shape=(seq_len, n_features))
gru1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(hidden_dim, name='latent_layer', **regularizers_GRU))(in1)
bn = tf.keras.layers.BatchNormalization()(gru1)
dense = tf.keras.layers.Dense(128, activation="relu", **regularizers)(bn)
drop = tf.keras.layers.Dropout(0.3)(dense)
d1   = tf.keras.layers.Dense(8, activation='softmax', **regularizers)(drop)

classifier_f_bi = tf.keras.Model(
    inputs=[in1], 
    outputs=[d1]
)

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
classifier_f_bi.compile(loss='categorical_crossentropy', optimizer=opt, 
    metrics=['accuracy'])

history_f_bi = classifier_f_bi.fit(
    x_train, y_train_encoded, batch_size=batch_size, epochs=epochs,
    validation_data=(x_val, y_val_encoded),
    callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-8)]
)

In [ ]:
plt.plot(history_f_bi.history['val_accuracy'], label='val_accuracy')
plt.plot(history_f_bi.history['accuracy'], label='accuracy')
plt.legend()